# PhononKit Interactive Transport Calculations

This notebook provides an interactive environment for testing phonon transport calculations without needing external configuration files. 

**Features:**
- Direct parameter modification in cells
- Quick visualization of results
- Easy testing of different electrode and scatter configurations
- Interactive parameter exploration

**Usage:**
1. Run all cells in order
2. Modify parameters in the configuration section
3. Re-run calculations to see updated results

In [ ]:
# Import Required Libraries
import sys
import json
import os
from unicodedata import name

import numpy as np
from model_systems import * 
import electrode as el
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.ticker import FormatStrFormatter
import tmoutproc as top
import calculate_kappa as ck
from utils import eigenchannel_utils as eu
from utils import constants as const

# Set up matplotlib for notebook display
%matplotlib inline
plt.style.use(['default'])  # Using default style for notebook

print("✓ All libraries imported successfully!")
print("✓ Ready for phonon transport calculations")

In [ ]:
class PhononTransport:
	"""Class for phonon transport calculations

	This class can be used for phonon transport calculations using a decimation technique to set up the electrodes. 
	Also describes the hessian matrix of the center part ab initio.
	"""

	def __init__(self, data_path, sys_descr, electrode_dict_L, electrode_dict_R, scatter_dict, E_D, M_L, M_R, M_C, N, T_min, T_max, kappa_grid_points):
		"""
		Args:
			electrode_dict (dict): Dictionary containing the configuration of the enabled electrode.
			scatter_dict (dict): Dictionary containing the configuration of the enabled scatter object.
			E_D (float): Debeye energy in meV.
			M_L (str): Atom type in the reservoir.
			M_C (str): Atom type coupled to the reservoir.
			N (int): Number of grid points.
			T_min (float): Minimum temperature for thermal conductance calculation.
			T_max (float): Maximum temperature for thermal conductance calculation.
			kappa_grid_points (int): Number of grid points for thermal conductance.
		"""

		self.data_path = data_path
		self.sys_descr = sys_descr
		self.electrode_dict_L = electrode_dict_L
		self.electrode_dict_R = electrode_dict_R
		self.scatter_dict = scatter_dict
		self.M_L = M_L
		self.M_C = M_C
		self.M_R = M_R
		self.N = N
		self.E_D = E_D

		self.temperature = np.linspace(T_min, T_max, kappa_grid_points)
		self.w = np.linspace(1E-3, self.E_D * 1.1, N)
		self.i = np.linspace(0, self.N, self.N, False, dtype=int)

		print("########## Setting up the scatter region ##########")
		self.scatter = self.__initialize_scatter(self.scatter_dict, self.electrode_dict_L, self.electrode_dict_R)
  
		print("########## Setting up the electrodes ##########")
		self.electrode_L = self.__initialize_electrode(self.electrode_dict_L)
		self.electrode_R = self.__initialize_electrode(self.electrode_dict_R)

		# Check for allowed combinations of electrode and scatter types
		if (self.electrode_dict_L["type"], self.electrode_dict_R["type"], self.scatter_dict["type"]) not in [
			("DebeyeModel", "DebeyeModel", "FiniteLattice2D"),
			("DebeyeModel", "DebeyeModel", "Chain1D"),
			("Ribbon2D", "Ribbon2D", "FiniteLattice2D"), 
			("Ribbon2D", "Ribbon2D", "Chain1D"),
			("Chain1D", "Chain1D", "Chain1D"),
			("InfiniteFourier2D", "InfiniteFourier2D", "FiniteLattice2D"),
			("InfiniteFourier2D", "InfiniteFourier2D", "Chain1D")
		]:
			raise ValueError(f"Invalid combination of electrode type '{self.electrode_L.type}', '{self.electrode_R.type}' and scatter type '{self.scatter.type}'")

		self.D = self.scatter.hessian
                  
		self.sigma_L, self.sigma_R = self.calculate_sigma()
		self.g_CC_ret, self.g_CC_adv = self.calculate_G_cc()
		self.T = self.calculate_transmission()
		self.kappa = self.calc_kappa()
	
	def __initialize_electrode(self, electrode_dict):
		"""Initializes the electrode based on the provided configuration."""
		
		match electrode_dict["type"]:

			case "DebeyeModel":
				return el.DebeyeModel(
					self.w,
					k_c = electrode_dict["k_x"],
					w_D = self.E_D
				)
			
			case "Chain1D":
				return el.Chain1D(
					self.w,
					interaction_range = electrode_dict["interaction_range"],
					interact_potential = electrode_dict["interact_potential"],
     				atom_type = electrode_dict["atom_type"],
					lattice_constant=electrode_dict["lattice_constant"],
     				k_x = electrode_dict["k_x"],
					k_c = electrode_dict["k_c"]
				)
			
			case "Ribbon2D":
				return el.Ribbon2D(
					self.w,
					interaction_range = electrode_dict["interaction_range"],
					interact_potential = electrode_dict["interact_potential"],
					atom_type = electrode_dict["atom_type"],
					lattice_constant = electrode_dict["lattice_constant"],
					N_y = electrode_dict["N_y"],
					N_y_scatter = self.scatter.N_y,
					M_L = self.M_L,
					M_C = self.M_C,
					k_x = electrode_dict["k_x"],
					k_y = electrode_dict["k_y"],
					k_xy = electrode_dict["k_xy"],
					k_c = electrode_dict["k_c"],
					k_c_xy = electrode_dict["k_c_xy"],
					left = electrode_dict["left"],
					right = electrode_dict["right"]
				)
			
			case "InfiniteFourier2D":
				return el.InfiniteFourier2D(
					self.w,
					interaction_range = electrode_dict["interaction_range"],
					interact_potential = electrode_dict["interact_potential"],
					atom_type = electrode_dict["atom_type"],
					lattice_constant=electrode_dict["lattice_constant"],
					N_q = electrode_dict["N_q"],
					k_x = electrode_dict["k_x"],
					k_y = electrode_dict["k_y"],
					k_xy = electrode_dict["k_xy"],
					k_c = electrode_dict["k_c"],
					k_c_xy = electrode_dict["k_c_xy"],
					N_y_scatter = self.scatter.N_y
				)
	
			case _:
				raise ValueError(f"Unsupported electrode type: {electrode_dict['type']}")

	def __initialize_scatter(self, scatter_dict, electrode_dict_l, electrode_dict_r):
		"""Initializes the scatter object based on the provided configuration."""
		match scatter_dict["type"]:

			case "FiniteLattice2D":
				return FiniteLattice2D(
					N_y = scatter_dict["N_y"],
					N_x = scatter_dict["N_x"],
					N_y_el_L = electrode_dict_l["N_y"],
					N_y_el_R = electrode_dict_r["N_y"],
					k_l_x = electrode_dict_l["k_x"],
					k_c_x = scatter_dict["k_x"],
					k_r_x = electrode_dict_r["k_x"],
					k_c_y = scatter_dict["k_y"],
					k_c_xy = scatter_dict["k_xy"],
					k_l_xy = electrode_dict_l["k_xy"],
					k_r_xy = electrode_dict_r["k_xy"],
					interact_potential = scatter_dict["interact_potential"],
					interaction_range = scatter_dict["interaction_range"],
					lattice_constant = scatter_dict["lattice_constant"],
					atom_type = scatter_dict["atom_type"]
				)
    
			case "Chain1D":
				return Chain1D(
					k_c = scatter_dict["k_x"],
					k_l = electrode_dict_l["k_x"],
					k_r = electrode_dict_r["k_x"],
					interact_potential = scatter_dict["interact_potential"],
					interaction_range = scatter_dict["interaction_range"],
					lattice_constant = scatter_dict["lattice_constant"],
					atom_type = scatter_dict["atom_type"],
					N = scatter_dict["N"]
				)
			
			case _:
				raise ValueError(f"Unsupported scatter type: {scatter_dict['type']}")

	def calculate_sigma(self):
		"""Calculates self energy"""
		match (self.electrode_dict_L["type"], self.electrode_dict_R["type"]):

			case ("DebeyeModel", "DebeyeModel"):
				g_L = self.electrode_L.g
				g_R = self.electrode_R.g
				k_c_l = self.electrode_L.k_c
				k_c_r = self.electrode_R.k_c
												
			case ("Chain1D", "Chain1D"):
				k_x = sum(self.electrode_L.ranged_force_constant()[0][i][1] for i in range(self.electrode_L.interaction_range))
				f_E = 0.5 * (self.w**2 - 2 * k_x - self.w * np.sqrt(self.w**2 - 4 * k_x, dtype=complex)) 

				sigma_L = np.zeros((self.N, self.scatter.N, self.scatter.N), dtype=complex)
				sigma_R = np.zeros((self.N, self.scatter.N, self.scatter.N), dtype=complex)

				for i in range(self.N):
					sigma_L[i, 0, 0] = f_E[i] 
					sigma_R[i, -1, -1] = f_E[i]
     
				return sigma_L, sigma_R
    			
			case ("Ribbon2D", "Ribbon2D"):
				g_L = self.electrode_L.g
				g_R = self.electrode_R.g
				# Simplified coupling matrix setup for notebook
				k_LC = np.zeros((2 * self.electrode_L.interaction_range * self.electrode_L.N_y, 
					 2 * self.scatter.N_x * self.scatter.N_y), dtype=float)
				k_RC = np.zeros((2 * self.electrode_R.interaction_range * self.electrode_R.N_y, 
					 2 * self.scatter.N_x * self.scatter.N_y), dtype=float)

			case ("InfiniteFourier2D", "InfiniteFourier2D"):
				g_L = self.electrode_L.g
				g_R = self.electrode_R.g
				k_LC = self.electrode_L.k_lc_LL
				k_RC = self.electrode_R.k_lc_LL

		# Initialize sigma array
		sigma_L = np.zeros((self.N, self.D.shape[0], self.D.shape[1]), dtype=complex)
		sigma_R = np.zeros((self.N, self.D.shape[0], self.D.shape[1]), dtype=complex)

		# DebeyeModel case
		if (g_L.shape, g_R.shape) == ((self.N,), (self.N,)): 
			sigma_nu_l = np.array(list(map(lambda i: k_c_l**2 * g_L[i], self.i)))
			sigma_nu_r = np.array(list(map(lambda i: k_c_r**2 * g_R[i], self.i)))

			match self.scatter_dict["type"]:
				case "FiniteLattice2D":
					for k in range(self.N):
						for n in range(self.scatter.N_y):
							sigma_L[k][n * 2, n * 2] = sigma_nu_l[k]
							sigma_R[k][sigma_R.shape[1] - 2 - n * 2, sigma_R.shape[2] - 2 - n * 2] = sigma_nu_r[k]
				
				case "Chain1D":
					for k in range(self.N):
						sigma_L[k][0, 0] = sigma_nu_l[k]
						sigma_R[k][-1, -1] = sigma_nu_r[k]

		return sigma_L, sigma_R

	def calculate_G_cc(self):
		"""Calculates Greens function for the central part"""
		g_CC_ret = np.array(list(map(lambda i: np.linalg.inv((self.w[i] + 1E-16j)**2 * np.identity(self.D.shape[0]) - self.D - self.sigma_L[i] - self.sigma_R[i]), self.i)))
		g_CC_adv = np.transpose(np.conj(g_CC_ret), (0, 2, 1))
		return g_CC_ret, g_CC_adv	

	def calculate_transmission(self):
		"""Calculates the transmission"""
		if self.electrode_dict_L["type"] == "Chain1D" and self.electrode_dict_R["type"] == "Chain1D" and self.scatter_dict["type"] == "Chain1D":
			# 1D Chain analytical solution
			k_x_L = sum(self.electrode_L.ranged_force_constant()[0][i][1] for i in range(self.electrode_L.interaction_range))
			k_x_R = sum(self.electrode_R.ranged_force_constant()[0][i][1] for i in range(self.electrode_R.interaction_range))
		
			g_E_L = np.where(4 * k_x_L - self.w**2 >= 0, self.w * np.sqrt(4 * k_x_L - self.w**2), 0)
			g_E_R = np.where(4 * k_x_R - self.w**2 >= 0, self.w * np.sqrt(4 * k_x_R - self.w**2), 0)
			
			lambda_L = np.zeros((self.N, self.scatter.N, self.scatter.N), dtype=complex)
			lambda_R = np.zeros((self.N, self.scatter.N, self.scatter.N), dtype=complex)
   
			for i in range(self.N):
				lambda_L[i, 0, 0] = g_E_L[i]
				lambda_R[i, -1, -1] = g_E_R[i]

			trans_prob_matrix = np.array(list(map(lambda i: np.dot(np.dot(self.g_CC_ret[i], lambda_L[i]), np.dot(self.g_CC_adv[i], lambda_R[i])), self.i)))
			tau_ph = np.array(list(map(lambda i: np.real(np.trace(trans_prob_matrix[i])), self.i)))
			return tau_ph
  
		# General case
		spectral_dens_L = 1j * (self.sigma_L - np.transpose(np.conj(self.sigma_L), (0, 2, 1)))
		spectral_dens_R = 1j * (self.sigma_R - np.transpose(np.conj(self.sigma_R), (0, 2, 1)))
		trans_prob_matrix = np.array(list(map(lambda i: np.dot(np.dot(self.g_CC_ret[i], spectral_dens_L[i]), np.dot(self.g_CC_adv[i], spectral_dens_R[i])), self.i)))
		tau_ph = np.array(list(map(lambda i: np.real(np.trace(trans_prob_matrix[i])), self.i)))
		return tau_ph

	def calc_kappa(self):
		"""Calculates the thermal conductance"""
		kappa = list()
		w_kappa = self.w * const.unit2SI
		E = const.h_bar * w_kappa
		E = E / const.har2J

		for j in range(0, len(self.temperature)):
			kappa.append(ck.calculate_kappa(self.T[1:len(self.T)], E[1:len(E)], self.temperature[j]) * const.har2pJ)
		
		return kappa

	def plot_transport_notebook(self):
		"""Simplified plotting for notebook"""
		# Filter extreme values
		mask = (self.T >= -50) & (self.T <= 50)
		w_filtered = self.w[mask]
		T_filtered = self.T[mask]

		fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
		
		# Transmission plot
		ax1.plot(w_filtered, T_filtered)
		ax1.set_yscale('log')
		ax1.set_xlabel('Phonon Energy (meV)')
		ax1.set_ylabel('Transmission')
		ax1.set_title('Phonon Transmission')
		ax1.grid(True)
		ax1.axhline(1, ls="--", color="red", alpha=0.7)
		
		# Thermal conductance plot
		ax2.plot(self.temperature, self.kappa)
		ax2.set_xlabel('Temperature (K)')
		ax2.set_ylabel('Thermal Conductance (pW/K)')
		ax2.set_title('Thermal Conductance')
		ax2.grid(True)
		
		plt.tight_layout()
		plt.show()
		
		print(f'Transmission: Max = {max(self.T):.3f}, Min = {min(self.T):.3f}')
		print(f'Thermal Conductance: Max = {max(self.kappa):.3f}, Min = {min(self.kappa):.3f} pW/K')

print("✓ PhononTransport class defined successfully!")

## Configuration Parameters

Modify these parameters to test different scenarios. Each configuration represents a different physical setup:

In [ ]:
# Configuration 1: Chain1D - Chain1D - Chain1D (Simple 1D case)
config_1d_chain = {
    "data_path": "./plot",
    "sys_descr": "1D_chain_test",
    "E_D": 20.0,  # Debye energy in meV
    "M_L": "C",
    "M_R": "C", 
    "M_C": "C",
    "N": 100,  # Number of frequency points
    "T_min": 1.0,
    "T_max": 300.0,
    "kappa_grid_points": 50,
    
    "electrode_dict_L": {
        "type": "Chain1D",
        "enabled": True,
        "interaction_range": 1,
        "interact_potential": "harmonic",
        "atom_type": "C",
        "lattice_constant": 1.0,
        "k_x": 100.0,
        "k_c": 100.0,
        "left": True,
        "right": False
    },
    
    "electrode_dict_R": {
        "type": "Chain1D", 
        "enabled": True,
        "interaction_range": 1,
        "interact_potential": "harmonic",
        "atom_type": "C",
        "lattice_constant": 1.0,
        "k_x": 100.0,
        "k_c": 100.0,
        "left": False,
        "right": True
    },
    
    "scatter_dict": {
        "type": "Chain1D",
        "enabled": True,
        "k_x": 100.0,
        "interact_potential": "harmonic", 
        "interaction_range": 1,
        "lattice_constant": 1.0,
        "atom_type": "C",
        "N": 2
    }
}

# Configuration 2: Debye Model with FiniteLattice2D
config_debye_2d = {
    "data_path": "./plot",
    "sys_descr": "debye_2D_test",
    "E_D": 25.0,
    "M_L": "C",
    "M_R": "C",
    "M_C": "C", 
    "N": 100,
    "T_min": 1.0,
    "T_max": 300.0,
    "kappa_grid_points": 50,
    
    "electrode_dict_L": {
        "type": "DebeyeModel",
        "enabled": True,
        "k_x": 50.0,
        "left": True,
        "right": False
    },
    
    "electrode_dict_R": {
        "type": "DebeyeModel",
        "enabled": True, 
        "k_x": 50.0,
        "left": False,
        "right": True
    },
    
    "scatter_dict": {
        "type": "FiniteLattice2D",
        "enabled": True,
        "N_y": 2,
        "N_x": 1,
        "k_x": 50.0,
        "k_y": 50.0,
        "k_xy": 10.0,
        "interact_potential": "harmonic",
        "interaction_range": 1,
        "lattice_constant": 1.0,
        "atom_type": "C"
    }
}

# Select which configuration to use (change this to switch between configurations)
current_config = config_1d_chain  # Change to config_debye_2d for 2D case

print("✓ Configuration loaded!")
print(f"✓ Selected: {current_config['sys_descr']}")
print(f"✓ Electrode types: {current_config['electrode_dict_L']['type']} - {current_config['scatter_dict']['type']} - {current_config['electrode_dict_R']['type']}")

## Initialize and Calculate

Run this cell to create the PhononTransport object and perform all calculations:

In [ ]:
# Initialize PhononTransport object with current configuration
try:
    PT = PhononTransport(
        data_path=current_config["data_path"],
        sys_descr=current_config["sys_descr"],
        electrode_dict_L=current_config["electrode_dict_L"],
        electrode_dict_R=current_config["electrode_dict_R"],
        scatter_dict=current_config["scatter_dict"],
        E_D=current_config["E_D"],
        M_L=current_config["M_L"],
        M_R=current_config["M_R"],
        M_C=current_config["M_C"],
        N=current_config["N"],
        T_min=current_config["T_min"],
        T_max=current_config["T_max"],
        kappa_grid_points=current_config["kappa_grid_points"]
    )
    
    print("✓ PhononTransport object created successfully!")
    print("✓ All calculations completed!")
    print(f"✓ System: {PT.sys_descr}")
    print(f"✓ Frequency grid: {PT.N} points from {PT.w[0]:.3f} to {PT.w[-1]:.3f} meV")
    print(f"✓ Temperature range: {PT.temperature[0]:.1f} to {PT.temperature[-1]:.1f} K")
    
except Exception as e:
    print(f"❌ Error initializing PhononTransport: {e}")
    print("Please check your configuration parameters.")

## Results Summary

Display key calculation results:

In [ ]:
# Display key results
if 'PT' in locals():
    print("=" * 50)
    print("PHONON TRANSPORT CALCULATION RESULTS")
    print("=" * 50)
    
    print(f"System Description: {PT.sys_descr}")
    print(f"Electrode Configuration: {PT.electrode_dict_L['type']} | {PT.scatter_dict['type']} | {PT.electrode_dict_R['type']}")
    print(f"Debye Energy: {PT.E_D} meV")
    print()
    
    print("TRANSMISSION PROPERTIES:")
    print(f"  • Maximum Transmission: {np.max(PT.T):.4f}")
    print(f"  • Minimum Transmission: {np.min(PT.T):.4f}")
    print(f"  • Mean Transmission: {np.mean(PT.T):.4f}")
    print(f"  • Transmission at E_D: {PT.T[int(PT.N * 0.9)]:.4f}")  # Approximate
    
    print()
    print("THERMAL CONDUCTANCE:")
    print(f"  • At 10K: {PT.kappa[int(len(PT.kappa)*0.03)]:.4f} pW/K")  # Approximate 10K
    print(f"  • At 100K: {PT.kappa[int(len(PT.kappa)*0.33)]:.4f} pW/K")  # Approximate 100K
    print(f"  • At 300K: {PT.kappa[-1]:.4f} pW/K")
    print(f"  • Maximum: {np.max(PT.kappa):.4f} pW/K")
    
    print()
    print("SYSTEM DIMENSIONS:")
    if hasattr(PT.scatter, 'N'):
        print(f"  • Scatter region size: {PT.scatter.N}")
    if hasattr(PT.scatter, 'N_y'):
        print(f"  • Scatter N_y: {PT.scatter.N_y}")
    if hasattr(PT.scatter, 'N_x'):
        print(f"  • Scatter N_x: {PT.scatter.N_x}")
    print(f"  • Hessian matrix size: {PT.D.shape}")
    
    print("=" * 50)
else:
    print("❌ Please run the initialization cell first!")

## Plot Transport Properties

Visualize transmission and thermal conductance:

In [ ]:
# Plot transport properties
if 'PT' in locals():
    # Filter out extreme values for better visualization
    mask = (PT.T >= -50) & (PT.T <= 50)
    w_filtered = PT.w[mask]
    T_filtered = PT.T[mask]
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Transmission vs Energy (linear scale)
    ax1.plot(w_filtered, T_filtered, 'b-', linewidth=2)
    ax1.set_xlabel('Phonon Energy (meV)')
    ax1.set_ylabel('Transmission')
    ax1.set_title('Phonon Transmission (Linear Scale)')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(1, ls="--", color="red", alpha=0.7, label='Unity')
    ax1.axvline(PT.E_D, ls="--", color="green", alpha=0.7, label=f'E_D = {PT.E_D} meV')
    ax1.legend()
    ax1.set_xlim(0, PT.E_D * 1.1)
    
    # 2. Transmission vs Energy (log scale)
    positive_mask = T_filtered > 0
    if np.any(positive_mask):
        ax2.semilogy(w_filtered[positive_mask], T_filtered[positive_mask], 'b-', linewidth=2)
        ax2.set_xlabel('Phonon Energy (meV)')
        ax2.set_ylabel('Transmission (log scale)')
        ax2.set_title('Phonon Transmission (Log Scale)')
        ax2.grid(True, alpha=0.3)
        ax2.axhline(1, ls="--", color="red", alpha=0.7, label='Unity')
        ax2.axvline(PT.E_D, ls="--", color="green", alpha=0.7, label=f'E_D = {PT.E_D} meV')
        ax2.legend()
        ax2.set_xlim(0, PT.E_D * 1.1)
    else:
        ax2.text(0.5, 0.5, 'No positive transmission values', ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Transmission (Log Scale) - No Data')
    
    # 3. Thermal Conductance vs Temperature
    ax3.plot(PT.temperature, PT.kappa, 'r-', linewidth=2)
    ax3.set_xlabel('Temperature (K)')
    ax3.set_ylabel('Thermal Conductance (pW/K)')
    ax3.set_title('Thermal Conductance vs Temperature')
    ax3.grid(True, alpha=0.3)
    
    # 4. Cumulative transmission integral
    cumulative_T = np.cumsum(T_filtered) * (w_filtered[1] - w_filtered[0]) if len(w_filtered) > 1 else np.array([0])
    ax4.plot(w_filtered, cumulative_T, 'g-', linewidth=2)
    ax4.set_xlabel('Phonon Energy (meV)')
    ax4.set_ylabel('Cumulative Transmission')
    ax4.set_title('Cumulative Transmission Integral')
    ax4.grid(True, alpha=0.3)
    ax4.axvline(PT.E_D, ls="--", color="green", alpha=0.7, label=f'E_D = {PT.E_D} meV')
    ax4.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Plots generated successfully!")
    print(f"✓ Filtered {len(PT.T) - len(T_filtered)} extreme transmission values")
else:
    print("❌ Please run the initialization cell first!")

## Plot Density of States (DOS)

Visualize the density of states for the electrodes (when available):

In [ ]:
# Plot Density of States (if available)
if 'PT' in locals():
    try:
        # Check if DOS data is available
        if hasattr(PT.electrode_L, 'dos') and hasattr(PT.electrode_R, 'dos'):
            dos_L = PT.electrode_L.dos
            dos_R = PT.electrode_R.dos
            
            # Check for coupled DOS if available
            dos_L_cpld = getattr(PT.electrode_L, 'dos_cpld', dos_L)
            dos_R_cpld = getattr(PT.electrode_R, 'dos_cpld', dos_R)
            dos_real_L_cpld = getattr(PT.electrode_L, 'dos_real_cpld', np.real(dos_L))
            dos_real_R_cpld = getattr(PT.electrode_R, 'dos_real_cpld', np.real(dos_R))
            
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
            
            # Left electrode DOS
            ax1.plot(PT.w, dos_real_L_cpld, 'b--', label='Re(DOS)', linewidth=2)
            ax1.plot(PT.w, np.imag(dos_L_cpld), 'r-', label='Im(DOS)', linewidth=2)
            ax1.set_xlabel('Phonon Energy (meV)')
            ax1.set_ylabel('DOS (a.u.)')
            ax1.set_title('Left Electrode DOS')
            ax1.grid(True, alpha=0.3)
            ax1.legend()
            ax1.set_xlim(0, PT.E_D * 1.1)
            
            # Right electrode DOS
            ax2.plot(PT.w, dos_real_R_cpld, 'b--', label='Re(DOS)', linewidth=2)
            ax2.plot(PT.w, np.imag(dos_R_cpld), 'r-', label='Im(DOS)', linewidth=2)
            ax2.set_xlabel('Phonon Energy (meV)')
            ax2.set_ylabel('DOS (a.u.)')
            ax2.set_title('Right Electrode DOS')
            ax2.grid(True, alpha=0.3)
            ax2.legend()
            ax2.set_xlim(0, PT.E_D * 1.1)
            
            # Combined DOS comparison
            ax3.plot(PT.w, np.imag(dos_L_cpld), 'b-', label='Left Im(DOS)', linewidth=2)
            ax3.plot(PT.w, np.imag(dos_R_cpld), 'r-', label='Right Im(DOS)', linewidth=2)
            ax3.set_xlabel('Phonon Energy (meV)')
            ax3.set_ylabel('Im(DOS) (a.u.)')
            ax3.set_title('DOS Comparison (Imaginary Part)')
            ax3.grid(True, alpha=0.3)
            ax3.legend()
            ax3.set_xlim(0, PT.E_D * 1.1)
            
            # DOS vs Transmission comparison
            mask = (PT.T >= -50) & (PT.T <= 50)
            w_filtered = PT.w[mask]
            T_filtered = PT.T[mask]
            
            ax4_twin = ax4.twinx()
            line1 = ax4.plot(w_filtered, T_filtered, 'g-', linewidth=2, label='Transmission')
            line2 = ax4_twin.plot(PT.w, np.imag(dos_L_cpld), 'b--', alpha=0.7, label='Left Im(DOS)')
            
            ax4.set_xlabel('Phonon Energy (meV)')
            ax4.set_ylabel('Transmission', color='g')
            ax4_twin.set_ylabel('Im(DOS) (a.u.)', color='b')
            ax4.set_title('Transmission vs DOS')
            ax4.grid(True, alpha=0.3)
            
            # Combine legends
            lines = line1 + line2
            labels = [l.get_label() for l in lines]
            ax4.legend(lines, labels, loc='upper right')
            
            plt.tight_layout()
            plt.show()
            
            print("✓ DOS plots generated successfully!")
            print(f"DOS Left electrode max/min: {np.max(dos_L):.4f}, {np.min(dos_L):.4f}")
            print(f"DOS Right electrode max/min: {np.max(dos_R):.4f}, {np.min(dos_R):.4f}")
            
        else:
            print("ℹ️ DOS data not available for current electrode configuration")
            print(f"Electrode types: {PT.electrode_dict_L['type']}, {PT.electrode_dict_R['type']}")
            print("DOS is typically available for Ribbon2D and InfiniteFourier2D electrodes")
            
    except Exception as e:
        print(f"⚠️ Error plotting DOS: {e}")
        print("DOS plotting may not be available for all electrode types")
        
else:
    print("❌ Please run the initialization cell first!")

## Interactive Parameter Testing

Quick parameter modifications for testing different scenarios. 

**How to use:**
1. Modify parameters in the cells below
2. Re-run the initialization cell 
3. Re-run the plotting cells to see updated results

In [ ]:
# Quick Parameter Modifications
# Modify these values and re-run initialization to see changes

print("Current Configuration:")
print(f"  • System: {current_config['sys_descr']}")
print(f"  • Debye Energy: {current_config['E_D']} meV")
print(f"  • Frequency points: {current_config['N']}")
print(f"  • Temperature range: {current_config['T_min']} - {current_config['T_max']} K")

# Example parameter modifications:
# Uncomment and modify these lines, then re-run initialization

# Change Debye energy
# current_config['E_D'] = 30.0  # Higher energy cutoff

# Change coupling strength  
# current_config['electrode_dict_L']['k_x'] = 150.0
# current_config['electrode_dict_R']['k_x'] = 150.0
# current_config['scatter_dict']['k_x'] = 150.0

# Change temperature range
# current_config['T_max'] = 500.0
# current_config['kappa_grid_points'] = 100

# Change frequency resolution
# current_config['N'] = 200  # Higher resolution

print("\n💡 Tip: Uncomment parameter modifications above to test different scenarios!")
print("📝 Remember to re-run the initialization cell after making changes.")

In [ ]:
# Quick Configuration Switching
# Run this cell to easily switch between predefined configurations

def switch_config(config_name):
    """Switch between predefined configurations"""
    global current_config
    
    if config_name == "1d_chain":
        current_config = config_1d_chain
        print("✓ Switched to 1D Chain configuration")
    elif config_name == "debye_2d":
        current_config = config_debye_2d  
        print("✓ Switched to Debye 2D configuration")
    else:
        print("❌ Unknown configuration. Available: '1d_chain', 'debye_2d'")
        return
    
    print(f"  • System: {current_config['sys_descr']}")
    print(f"  • Electrodes: {current_config['electrode_dict_L']['type']} - {current_config['scatter_dict']['type']} - {current_config['electrode_dict_R']['type']}")
    print("  • Remember to re-run the initialization cell!")

# Usage examples:
# switch_config("1d_chain")    # Switch to 1D chain configuration
# switch_config("debye_2d")    # Switch to Debye 2D configuration

print("💡 Use switch_config('1d_chain') or switch_config('debye_2d') to quickly change configurations")

## Notes and Tips

**🚀 Quick Start Workflow:**
1. Run all cells in order for the first time
2. Modify parameters in the configuration section
3. Re-run initialization cell
4. Re-run plotting cells

**🔧 Parameter Guide:**
- `E_D`: Debye energy cutoff (meV) - affects frequency range
- `k_x`, `k_y`, `k_xy`: Force constants - control coupling strength
- `N`: Number of frequency points - affects calculation resolution
- `interaction_range`: Number of nearest neighbors for interactions
- `T_min`, `T_max`: Temperature range for thermal conductance

**⚠️ Important Notes:**
- Some electrode combinations may not be implemented
- Very large N values will slow down calculations
- Extreme parameter values may cause numerical instability
- DOS plots only work with certain electrode types

**🐛 Troubleshooting:**
- If initialization fails, check electrode/scatter type compatibility
- For numerical issues, try reducing E_D or force constants
- Memory errors: reduce N (frequency points)
- Missing plots: ensure the PT object was created successfully